### Initially, the comments were written in Russian. Translated by Google Translator

### Изначально комментарии были написаны на русском языке. Переведены через Google-переводчик

# Task:

Using an SQL query, extract data from the database about customers who have bought more than one course.
Based on this data, make a table of recommendations for purchases of the following courses.

# Задача:

При помощи SQL-запроса извлечь из базы данных данные о клиентах, купивших более одного курса.
Составить на основе этих данных таблицу рекомендаций для покупок следующих курсов.

In [1]:
# importing the main libraries and functions
# импортируем основные библиотеки и функции
import pandas as pd
import numpy as np
import itertools as it
from collections import Counter as ct
import psycopg2
import psycopg2.extras

In [2]:
# We request the data we are interested in using an SQL query
# Запрашиваем интересующие нас данные через SQL
def getData():
    query = '''with t_1 as 
(select     
*
from final.cart_items ci left join  final.carts ct on ci.cart_id=ct.id
where ct.state = 'successful' and ci.resource_type = 'Course' 
),      

t_2 as
(select
user_id,
count(distinct resource_id) cours_count
from t_1 
group by 1
having count(distinct resource_id) > 1
)

select
t_2.user_id,
resource_id
from t_2 left join t_1 on t_2.user_id=t_1.user_id'''
    conn = psycopg2.connect(dbname='skillfactory',
                            user='skillfactory',
                            host='84.201.134.129',
                            password='cCkxxLVrDE8EbvjueeMedPKt',
                            port=5432)
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    dict_cur.close()
    conn.close()
    return data
cours_df=pd.DataFrame(getData())

 # check the correctness of the received data
 # проверяем корректность полученых данных
cours_df.head()

,user_id,resource_id
0,1010882,490
1,1010802,514
2,1120298,514
3,750528,552
4,191574,523


In [3]:
# we group data by users, for each user we create a list of courses purchased by them
# групперуем данные по пользователям, для каждого пользователя создаём список купленых им курсов
group_cours_df = cours_df.groupby(['user_id'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
group_cours_df.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [4]:
# using the combinations function from the itertools module, we create pairs of courses purchased by one user
# при помощи функции combinations из модуля itertools, создаём пары курсов, купенные одним пользователем
list_cours = list()
for x in group_cours_df['resource_id']:
    for p in it.combinations(x,2):
        list_cours.append(p)
len(list_cours)

40017

In [5]:
# using the Counter function, we create a list of all unique pairs and the number of their repetitions
# с помощью функции Counter создаём список всех уникальных пар и количество их повторений
count_list_cours = ct(list_cours)

# we create a dataframe based on the list. The first column is a pair, the second is the number of repetitions of the pair
# на основе списка создаём датафрейм. Первый столбец - пара, второй количество повторений пары
cours_pair_df = pd.DataFrame.from_dict(count_list_cours, orient='index').reset_index()
cours_pair_df.columns = ['pair','count_purc']

# checking the resulting dataframe
# проверяем получившийся датафрейм
cours_pair_df.describe()

,count_purc
count,3989.000000
mean,10.031838
std,26.355998
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,797.000000


In [6]:
# for recommendations, we take the upper quartile
# using the numpy library and the percentile function, we select the most popular 25% of pairs, write them into the dictionary
# для рекомендаций возмём верхний квартиль
# при помощи библиотеки numpy  и функции percentile отбираем самые популярные 25% пар, записываем их в словарь
percent = np.percentile(cours_pair_df['count_purc'],75)
popular_pair_dict = {k: v for k, v in count_list_cours.items() if v > percent  }

In [7]:
# creating the recommend auxiliary function 
# iterates over the keys of the popular_pair_dict dictionary, selecting a suitable recommendation
# создаём вспомогательную функцию recommend 
# перебирает ключи словаря popular_pair_dict, подбирая подходящую рекомендацию
def recommend (course):
    x = []
    for i in popular_pair_dict.keys():
        if i [0] == course:
            x.append((i,popular_pair_dict[i]))
    scl = sorted( x, key=lambda x: x[1], reverse = True)
    return scl[:2]

In [8]:
# creating a recommendation for courses for which there was no popular pair, we take the most popular pair of courses for this
# создаём рекомендацию для курсов, к которым не нашлось популярной пары, берём для этого самую популярную пару курсов
is_not_pair_1 = cours_pair_df.sort_values('count_purc', ascending= False).iloc[0][0][0]
is_not_pair_2 = cours_pair_df.sort_values('count_purc', ascending= False).iloc[0][0][1]

In [9]:
# creating a unique list of all courses
# создаём уникальный список всех курсов
courses=cours_df['resource_id'].unique()

In [10]:
# creating an empty recommendation table
# создаём пустую таблицу рекомендаций
advice_df = pd.DataFrame(columns = ['recommendation_1','recommendation_2'])

# using the recommend function, we fill in this table, filling in the empty values with the course from the most popular pair
# с помощью функции recommend заполняем эту таблицу, заполняя пустые значения курсом из самой популярной пары
for i in courses:
    if len(recommend(i)) == 2:
        advice_df.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]]
    elif len(recommend(i)) == 1:
        advice_df.loc[i] = [recommend(i)[0][0][1], is_not_pair_2]
    else:
        advice_df.loc[i] = [is_not_pair_1, is_not_pair_2]

In [11]:
# moving the course number from the index to a separate column
# переносим номер курса из индекса в отдельную колонку
advice_df=advice_df.reset_index()
advice_df=advice_df.rename(columns = {'index' : 'course'})

In [15]:
# creating a script to search for recommendations by the number of the entered course
# создаём скрипт для поиска рекомендаций по номеру введённого курса
course_search = int(input('Номер курса  '))
if course_search in courses:
    info = advice_df[advice_df['course'] == course_search]
    print('Рекомендуем курсы', info.iloc[0][1],'и',info.iloc[0][2])
else:
    print('Курс не найден(')

Номер курса  735
Курс не найден(


In [ ]:
#  record the resulting dataframe as a csv file
# записываем получившийся датафрейм в виде csv-файла
advice_df.to_csv('advice_df.csv')